In [1]:
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Load in the Validation and Test Set

In [2]:
X_val = np.load("/Users/Hoang/Machine_Learning/skin_cancer/skin_cancer_192_256/256_192_val.npy")

In [3]:
X_test = np.load("/Users/Hoang/Machine_Learning/skin_cancer/skin_cancer_192_256/256_192_test.npy")

In [4]:
y_val = np.load("/Users/Hoang/Machine_Learning/skin_cancer/skin_cancer_192_256/val_labels.npy")

In [5]:
y_test = np.load("/Users/Hoang/Machine_Learning/skin_cancer/skin_cancer_192_256/test_labels.npy")

In [6]:
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [7]:
X_val.shape, X_test.shape, y_val.shape, y_test.shape

((902, 192, 256, 3), (1002, 192, 256, 3), (902, 7), (1002, 7))

In [9]:
input_shape = X_val[0,:,:,:].shape
model_input = Input(shape=input_shape)

## Define InceptionV3

In [16]:
inception = InceptionV3(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [17]:
for layer in inception.layers:
    layer.trainable = True

In [18]:
inception_last_layer = inception.get_layer('mixed10')
print('last layer output shape:', inception_last_layer.output_shape)
inception_last_output = inception_last_layer.output

last layer output shape: (None, 4, 6, 2048)


In [19]:
# Flatten the output layer to 1 dimension
x_inception = layers.GlobalMaxPooling2D()(inception_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_inception = layers.Dense(512, activation='relu')(x_inception)
# Add a dropout rate of 0.7
x_inception = layers.Dropout(0.5)(x_inception)
# Add a final sigmoid layer for classification
x_inception = layers.Dense(7, activation='softmax')(x_inception)

# Configure and compile the model

inception_model = Model(model_input, x_inception)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
inception_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [20]:
inception_model.load_weights("InceptionV3full.h5")

In [21]:
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 192, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_185 (Conv2D)             (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_185 (BatchN (None, 95, 127, 32)  96          conv2d_185[0][0]                 
__________________________________________________________________________________________________
activation_184 (Activation)     (None, 95, 127, 32)  0           batch_normalization_185[0][0]    
__________________________________________________________________________________________________
conv2d_186

## Define DenseNet 201 

In [22]:
denseNet = DenseNet201(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [23]:
for layer in denseNet.layers:
    layer.trainable = True

In [24]:
denseNet_last_layer = denseNet.get_layer('relu')
print('last layer output shape:', denseNet_last_layer.output_shape)
denseNet_last_output = denseNet_last_layer.output

last layer output shape: (None, 6, 8, 1920)


In [25]:
# Flatten the output layer to 1 dimension
x_denseNet = layers.GlobalMaxPooling2D()(denseNet_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_denseNet = layers.Dense(512, activation='relu')(x_denseNet)
# Add a dropout rate of 0.7
x_denseNet = layers.Dropout(0.5)(x_denseNet)
# Add a final sigmoid layer for classification
x_denseNet = layers.Dense(7, activation='softmax')(x_denseNet)

# Configure and compile the model

denseNet_model = Model(model_input, x_denseNet)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
denseNet_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [26]:
denseNet_model.load_weights("DenseNetFull.h5")

In [27]:
denseNet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 192, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 198, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 96, 128, 64)  9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 96, 128, 64)  256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

## Define Ensemble Model

In [29]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = layers.Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [30]:
ensemble_model = ensemble([denseNet_model, inception_model], model_input)

In [32]:
ensemble_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## Testing

This is taking very long because I am running on my own PC to save money instead of on Floydhub

In [33]:
loss_val, acc_val = ensemble_model.evaluate(X_val, y_val, verbose=1)

902/902 [==============================] - 368s 408ms/step


In [34]:
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

Validation: accuracy = 0.888027  ;  loss_v = 0.399892


In [35]:
loss_test, acc_test = ensemble_model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

1002/1002 [==============================] - 419s 418ms/step
Test: accuracy = 0.885230  ;  loss = 0.411560
